Example of the process of doing DEMs for all orbits for a given observation of an AR – where we don't window out any intervals because of shifts, etc. Instead, we will save energy-specific information about the percent of NuSTAR emission in the chosen region as a function of time, for later inspection. This will help identify cases where pointing shifts might cause unphysical distortion to the DEM inputs. 

Note: auto-download of an AIA file (for use in the initial co-alignmnet) will break while the JSOC is still down. We will add a thing where you can point to an existing AIA file instead.

Overview:

- Define orbits
- Run time interval selection
- Examine resulting intervals
- Manually establish a co-alignment shift between NuSTAR and AIA
- Automatically find co-alignment shifts + make regions for all other time intervals (note: this relies on the assumption that the COM is a good representation of the location of the brightest source, i.e. that the NuSTAR data is primarially one blob).
- Save AIA region files for NCCS input
- NOT IN THIS NOTEBOOK: YOU THEN TAKE THOSE AND MAKE AIA INPUTS ON THE NCCS
- Conduct AIA/NuSTAR DEMs as a function of time, given all the above
- Plot results.
- Print some stats about "left out" times.

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# import glob
# from astropy.io import fits
# from astropy import units as u
# import importlib
# import pathlib

#Path to top-level do-dem directory - edit for your system.
path_to_dodem = '/Users/jmdunca2/do-dem/'
from sys import path as sys_path
sys_path.append(path_to_dodem+'/dodem/')

# # #import nustar_dem_prep as nu
# import initial_analysis as ia
# import orbit_auto as oa
# import time_interval_selection as tis
# import nustar_utilities as nuutil
# import gauss2D as g2d
import all_nu_analysis as ana
import orbit_auto as oa
import visualize_dem_results as viz


import pickle
import pathlib
import importlib
import numpy as np
import subprocess


In [ ]:
importlib.reload(ana)
key = '06-jun-20'

#===============================================================
#Update dictionary to have correct aia path for NCCS-prepped data.
#===============================================================
with open('all_targets.pickle', 'rb') as f:
    data = pickle.load(f)

ARDict = data[key]

id_dirs = ARDict['datapaths']
obsids = ARDict['obsids']
working_dir = ARDict['working_dir']

ARDict['prepped_aia'] = working_dir+'all_aia_dicts_'+key+'_post/'

data[key] = ARDict

with open('all_targets.pickle', 'wb') as f:
         # Pickle the 'data' dictionary using the highest protocol available.
         pickle.dump(data, f, pickle.HIGHEST_PROTOCOL) 

#===============================================================
#DO DEMS

import initial_analysis as ia
import nustar_utilities as nuutil

for id in id_dirs:
    evt_data, hdr = ia.return_submap(datapath=id, fpm='A', return_evt_hdr=True)
    time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
    timerange = [time0, time1]
    print(timerange[0].strftime('%H-%M-%S'), timerange[1].strftime('%H-%M-%S'))

ana.do_key_dem(key, missing_last=True, missing_orbit=1)

#===============================================================


#===============================================================
#Copy DEM result files to common directory


minT=5.6
maxT=7.2

all_time_intervals, all_time_intervals_list = oa.find_all_intervals(working_dir, shush=True, 
                                                                missing_last=False)

for time in all_time_intervals_list:
    timestring = time[0].strftime('%H-%M-%S')
    stopstring = time[1].strftime('%H-%M-%S')
    timestring=timestring+'_'+stopstring
    
    demfile = working_dir+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
    print(demfile)

    status = subprocess.call('cp '+demfile+' ./compact_results/', shell=True) 

#===============================================================



In [ ]:
importlib.reload(ana)
key = '29-may-18_1'

#===============================================================
#Update dictionary to have correct aia path for NCCS-prepped data.
#===============================================================
with open('all_targets.pickle', 'rb') as f:
    data = pickle.load(f)

ARDict = data[key]

id_dirs = ARDict['datapaths']
obsids = ARDict['obsids']
working_dir = ARDict['working_dir']

ARDict['prepped_aia'] = working_dir+'all_aia_dicts_'+key+'_post/'

data[key] = ARDict

with open('all_targets.pickle', 'wb') as f:
         # Pickle the 'data' dictionary using the highest protocol available.
         pickle.dump(data, f, pickle.HIGHEST_PROTOCOL) 

#===============================================================


ana.do_key_dem(key)


# #===============================================================
# #Copy DEM result files to common directory


minT=5.6
maxT=7.2

all_time_intervals, all_time_intervals_list = oa.find_all_intervals(working_dir, shush=True, 
                                                                missing_last=False)

for time in all_time_intervals_list:
    timestring = time[0].strftime('%H-%M-%S')
    stopstring = time[1].strftime('%H-%M-%S')
    timestring=timestring+'_'+stopstring

    demfile = working_dir+timestring+'/'+timestring+'_'+str(minT)+'_'+str(maxT)+'_'+key+'_MC_DEM_result.pickle'
    print(demfile)

    status = subprocess.call('cp '+demfile+' ./compact_results/', shell=True) 

#===============================================================

In [ ]:

# minT=5.6
# maxT=7.2
# name=key

# all_time_intervals, all_time_intervals_list = oa.find_all_intervals(working_dir, shush=True, 
#                                                                     missing_last=False)



# time_intervals = all_time_intervals_list

# importlib.reload(viz)
# vals = viz.get_DEM_timeseries(time_intervals, working_dir, minT, maxT, name)    


# peaks=vals['peaks']
# peaksmk = [10**m1/1e6 for m1 in peaks]    

# backcolors=['pink', 'lavenderblush']
# color='Red'
    
# viz.pretty_orbit_timeseries(time_intervals, peaksmk, 'DEM Peak Temperature (MK)', 'DEM Peak Temperature',
#                         color, backcolors, working_dir=working_dir)


# backcolors=['powderblue', 'aliceblue']
# color='Blue'

# above10s=np.array(vals['above10s'])
# above10s_=above10s[:,0]

# viz.pretty_orbit_timeseries(time_intervals, above10s_, 'EM (cm^-5)', 'Total EM >10 MK',
#                         color, backcolors, error=True, quantity_low=above10s[:,1], quantity_high=above10s[:,2], 
#                         ylog=True, comparisonbar=True, comp_band=[1.8e22, 1.5e23, 'Ishikawa (2017) 95%'],
#                             working_dir=working_dir)

# backcolors=['powderblue', 'aliceblue']
# color='Green'

# above7s=np.array(vals['above7s'])
# above7s_=above7s[:,0]

# viz.pretty_orbit_timeseries(time_intervals, above7s_, 'EM (cm^-5)', 'Total EM >7 MK',
#                         color, backcolors, error=True, quantity_low=above7s[:,1], quantity_high=above7s[:,2], 
#                         ylog=True, working_dir=working_dir)

# backcolors=['powderblue', 'aliceblue']
# color='Purple'

# above5s=np.array(vals['above5s'])
# above5s_=above5s[:,0]

# viz.pretty_orbit_timeseries(time_intervals, above5s_, 'EM (cm^-5)', 'Total EM >5 MK',
#                         color, backcolors, error=True, quantity_low=above5s[:,1], quantity_high=above5s[:,2], 
#                         ylog=True, working_dir=working_dir)


# backcolors=['khaki', 'lemonchiffon']
# color='Orange'

# val=np.array(vals['low_powers'])

# viz.pretty_orbit_timeseries(time_intervals, val, 'Index', 'Lower Power Law',
#                         color, backcolors, error=False, working_dir=working_dir)


# backcolors=['khaki', 'lemonchiffon']
# color='Red'

# val=np.array(vals['hi_powers'])*-1

# viz.pretty_orbit_timeseries(time_intervals, val, 'Index', 'Upper Power Law',
#                         color, backcolors, error=False, working_dir=working_dir)



In [ ]:
keys = ['09-sep-18', '10-sep-18', '12-apr-19', '13-apr-19', '06-jun-20', '29-apr-21', '03-may-21_1', '03-may-21_2', '29-may-18_1']

In [ ]:
import glob
importlib.reload(viz)

import pandas as pd
import astropy.time

df = pd.read_csv('fpmA.csv')
starts = df['flare_start'].values
stops = df['flare_end'].values

from astropy import units as u
early_starts = [(astropy.time.Time(s)-2*u.min) for s in starts]
late_stops = [(astropy.time.Time(s)+2*u.min) for s in stops]

res_files = glob.glob('./compact_results/*')
res_files.sort()

all_above10s_flares = []
all_above10s_non = []
all_above10s=[]

for f in res_files:
    flare=False

    data, timestring, time = viz.load_DEM(f)

    b4 = [s < time[0] for s in early_starts]
    ea = [s > time[0] for s in late_stops]
    es = np.where(np.logical_and(b4, ea))

    if es[0].size > 0:
        flare=True

    b4 = [s > time[0] for s in early_starts]
    ea = [s < time[1] for s in early_starts]
    es = np.where(np.logical_and(b4, ea))

    if es[0].size > 0:
        flare=True
    
    res = viz.get_DEM_params(f)

    m1, max1, above5_, above7_, above10_, \
        above_peak, below_peak, above_635, below_635, \
           data['chanax'], data['dn_in'], data['edn_in'], \
                powerlaws, EMT_all, EMT_thresh = res

    if flare:
        all_above10s_flares.extend(above10_)
    else:
        all_above10s_non.extend(above10_)

    all_above10s.extend(above10_)



In [ ]:
area_i = 100**2
area_m = np.pi*150**2
print(area_i, area_m)
factor = area_m/area_i
factor = 1

In [ ]:
from matplotlib import pyplot as plt 

logbins = np.geomspace(np.min(all_above10s), np.max(all_above10s), 50)

fig, ax = plt.subplots(figsize=(15,4), tight_layout = {'pad': 1})

ax.hist(all_above10s, bins=logbins, color='green', edgecolor='black')
ax.hist(np.array(all_above10s_non)*factor, bins=logbins, color='skyblue', edgecolor='black', label='Non-flare time bins')
ax.hist(np.array(all_above10s_flares)*factor, bins=logbins, color='purple', edgecolor='black', label="Bins during Reed's flares")
ax.set_xscale('log')
ax.axvline(1.8e22, color='Red')
ax.axvline(1.5e23, color='Red')
ax.axvspan(1.8e22, 1.5e23, alpha=0.3, color='Red', label='Ishikawa (2017) 95% Interval')
ax.set_ylabel('Number of intervals')
ax.set_xlabel('EM Integrated >10 MK')
ax.legend()

In [ ]:
done_sources = {k: all_targets[k] for k in keys}
ana.get_exposures(done_sources, dogoes=False)

In [ ]:
import pandas as pd

df = pd.read_csv('fpmA.csv')
starts = df['flare_start'].values
stops = df['flare_end'].values

for i in range(0, len(starts)-1):
    print(starts[i], stops[i])

In [ ]:
for f in flaretimes:
    print(f[0], f[1])

In [ ]:
flaretimes=[]
nonflaretimes=[]
for time in all_time_intervals_list:
    b4 = [s < time[0] for s in starts]
    ea = [s > time[0] for s in stops]
    es = np.where(np.logical_and(b4, ea))

    if es[0].size > 0:
        #print(starts[es], stops[es])
        #print(time)
        flaretimes.append(time)
        continue

    b4 = [s > time[0] for s in starts]
    ea = [s < time[1] for s in starts]
    es = np.where(np.logical_and(b4, ea))

    if es[0].size > 0:
        #print(starts[es], stops[es])
        #print(time)
        flaretimes.append(time)
        continue

    nonflaretimes.append(time)

For the time interval to overlap with a flare, it could either ENVELOP a flare, start before (and end during) start during (and end during) and start + end within the flare. 

- fstart tstart tend fend - ENVELOP
- fstart tstart fend tend - STOPSIDE
  
- tstart fstart tend fend - STARTSIDE
- tstart fstart fend tend - DURING



In [ ]:
from matplotlib import pyplot as plt 

logbins = np.geomspace(np.min(all_above10s), np.max(all_above10s), 30)

fig, ax = plt.subplots(figsize=(15,4), tight_layout = {'pad': 1})

ax.hist(all_above10s, bins=logbins, color='skyblue', edgecolor='black')
ax.set_xscale('log')
ax.axvline(1.8e22, color='Red')
ax.axvline(1.5e23, color='Red')
ax.axvspan(1.8e22, 1.5e23, alpha=0.3, color='Red', label='Ishikawa (2017) 95% Interval')
ax.set_ylabel('Number of intervals')
ax.set_xlabel('EM Integrated >10 MK')
ax.legend()

In [ ]:
# import glob

# #Set path to obsid directory - initial pipeline should have been run already.
# ind=0
# datapath=id_dirs[ind]
# obsid=obsids[ind]

# evt_data, hdr = ia.return_submap(datapath=datapath, fpm='A', return_evt_hdr=True)
# time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
# timerange = [time0.tt.datetime, time1.tt.datetime]
# from datetime import timezone
# timerange = [t.replace(tzinfo=timezone.utc) for t in timerange]

# #Comment second line if you're not using this same example nustar orbit
# #Edit it to include only the desired time interval (default- all times in file) once you've run this once
# #timerange=[]
# #timerange=[datetime.datetime(2018, 5, 29, 22, 22), datetime.datetime(2018, 5, 29, 23, 20)]

# evtA = glob.glob(datapath+'/event_cl/*A06_cl.evt')
# evtB = glob.glob(datapath+'/event_cl/*B06_cl.evt')
# hkA  = glob.glob(datapath+'/hk/*A_fpm.hk')
# hkB  = glob.glob(datapath+'/hk/*B_fpm.hk')

# import lightcurves as lc

# importlib.reload(lc)
# lc.prepare_nustar_lightcurves(evtA, evtB, hkA, hkB, timebin=15, erange=[2.,4.], 
#                               livetime_corr=False, save_dir=working_dir)
# lc.prepare_nustar_lightcurves(evtA, evtB, hkA, hkB, timebin=15, erange=[4.,6.], 
#                               livetime_corr=False, save_dir=working_dir)
# lc.prepare_nustar_lightcurves(evtA, evtB, hkA, hkB, timebin=15, erange=[6.,10.], 
#                               livetime_corr=False, save_dir=working_dir)

# lc.plot_nustar_lightcurves(eranges = [[2.,4.],[4.,6.],[6.,10.]],
#                            timerange=timerange, save_dir=working_dir)

In [ ]:
# importlib.reload(oa)
# oa.check_conseq(working_dir)